In [21]:
import numpy as np
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 

import requests
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import folium
from sklearn.neighbors import KNeighborsClassifier
print('Libraries imported.')

Libraries imported.


In [22]:
df_Paris = pd.read_csv(r'D:\Study\Final_Assignment\laposte_hexasmal.csv',sep=';')
df1 = df_Paris[df_Paris['Nom_commune'].str.contains('PARIS ')]
df1.drop(['Code_commune_INSEE'],axis=1,inplace=True)
df1.drop(['Ligne_5'],axis=1,inplace=True)
df1.drop(['Libellé_d_acheminement'],axis=1,inplace=True)
df = df1.reset_index(drop=True)
df.drop([7],axis=0,inplace=True)
df[['longtitude', 'latitude']] = df['coordonnees_gps'].str.split(',', 2, expand = True)

C:\Users\jyzhang\Anaconda3\lib\site-packages\pandas\core\frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [23]:
df.drop(['coordonnees_gps'],axis=1,inplace=True)
Paris = df
Paris.rename(columns={'Nom_commune':'Neighborhood'},inplace=True)
Paris=Paris.reset_index(drop=True)


In [24]:
df_London = pd.read_excel('D:\Study\Final_Assignment\London.xlsx')
df_London.rename(columns={'Borough':'Neighborhood'},inplace=True)
df_London.rename(columns={'longitude':'longtitude'},inplace=True)
df_London.head()

,Neighborhood,Population (2013 est)[1],longtitude,latitude
0,Barking and Dagenham [note 1],194352,51.5607,0.1557
1,Barnet,369088,51.6252,-0.1517
2,Bexley,236687,51.4549,0.1505
3,Brent,317264,51.5588,-0.2817
4,Bromley,317899,51.4039,0.0198


In [25]:
df = pd.concat([Paris,df_London],axis=0)
df1 = df.reset_index(drop=True)
df1.drop(['Code_postal'],axis=1,inplace=True)
df1.drop(['Population (2013 est)[1]'],axis=1,inplace=True)
df1

,Neighborhood,longtitude,latitude
0,PARIS 02,48.8679033789,2.34410716666
1,PARIS 03,48.8630541318,2.35936105897
2,PARIS 08,48.8725272666,2.31258256042
3,PARIS 14,48.8289932116,2.32710088326
4,PARIS 16,48.8603987604,2.2620995594
5,PARIS 17,48.8873371665,2.30748555949
6,PARIS 19,48.886868623,2.38469432787
7,PARIS 06,48.8489680919,2.33267089859
8,PARIS 12,48.8351562307,2.41980703497
9,PARIS 13,48.8287176845,2.36246822852


In [26]:
CLIENT_ID = 'USQRMYMSFBQ0Z4SF0KXZDWINR55CTA3VCPNHTNMYOMH0NVLB' 
CLIENT_SECRET = 'HKW2GKHKR1W23LK1J11L0OCGIS1WZD4ZBNNM2Y4WQ0SYOP3I' 
VERSION = '20180605' 
LIMIT = 100

In [27]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [28]:
venues = getNearbyVenues(names=df1['Neighborhood'],
                                   latitudes=df1['longtitude'],
                                   longitudes=df1['latitude']
                                  )

PARIS 02
PARIS 03
PARIS 08
PARIS 14
PARIS 16
PARIS 17
PARIS 19
PARIS 06
PARIS 12
PARIS 13
PARIS 18
PARIS 01
PARIS 05
PARIS 09
PARIS 10
PARIS L HOPITAL
PARIS 04
PARIS 07
PARIS 11
PARIS 15
PARIS 16
PARIS 20
Barking and Dagenham [note 1]
Barnet
Bexley
Brent
Bromley
Camden
Croydon
Ealing
Enfield
Greenwich [note 2]
Hackney
Hammersmith and Fulham [note 4]
Haringey
Harrow
Havering
Hillingdon
Hounslow
Islington
Kensington and Chelsea
Kingston upon Thames
Lambeth
Lewisham
Merton
Newham
Redbridge
Richmond upon Thames
Southwark
Sutton
Tower Hamlets
Waltham Forest
Wandsworth
Westminster


In [29]:
print(venues.shape)
venues.head()

(2819, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,PARIS 02,48.8679033789,2.34410716666,Pizzeria Popolare,48.868074,2.343379,Pizza Place
1,PARIS 02,48.8679033789,2.34410716666,Lockwood,48.867727,2.346945,Cocktail Bar
2,PARIS 02,48.8679033789,2.34410716666,Nose,48.866242,2.344885,Perfume Shop
3,PARIS 02,48.8679033789,2.34410716666,L'Appartement Sézane,48.869574,2.345060,Women's Store
4,PARIS 02,48.8679033789,2.34410716666,Le Moderne,48.868856,2.342142,French Restaurant


In [30]:
venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Barking and Dagenham [note 1],8,8,8,8,8,8
Barnet,5,5,5,5,5,5
Bexley,29,29,29,29,29,29
Brent,78,78,78,78,78,78
Bromley,40,40,40,40,40,40
Camden,79,79,79,79,79,79
Croydon,39,39,39,39,39,39
Ealing,75,75,75,75,75,75
Enfield,51,51,51,51,51,51


In [31]:
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

onehot['Neighborhood'] = venues['Neighborhood'] 

fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]
onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Non-Profit,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Resort,Restaurant,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoothie Shop,Snack Place,Soba Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Train Station,T

In [32]:
grouped = onehot.groupby('Neighborhood').mean().reset_index()
grouped.head()

,Neighborhood,Afghan Restaurant,African Restaurant,Airport,Airport Lounge,Airport Service,American Restaurant,Antique Shop,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boarding House,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Station,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Chaat Place,Champagne Bar,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Creperie,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Donut Shop,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,English Restaurant,Escape Room,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food & Drink Shop,Food Court,Food Stand,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gelato Shop,General College & University,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Chinese Restaurant,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Kebab Restaurant,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Lyonese Bouchon,Malay Restaurant,Market,Martial Arts School,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Non-Profit,Noodle House,Okonomiyaki Restaurant,Optical Shop,Organic Grocery,Outdoor Event Space,Outdoor Sculpture,Outdoor Supply Store,Outlet Mall,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Persian Restaurant,Peruvian Restaurant,Pharmacy,Pizza Place,Platform,Playground,Plaza,Polish Restaurant,Pool,Portuguese Restaurant,Print Shop,Pub,Public Art,Rafting,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Resort,Restaurant,Road,Romanian Restaurant,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shanxi Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Smoothie Shop,Snack Place,Soba Restaurant,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Street Art,Street Food Gathering,Strip Club,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Train Station,T

In [33]:
num_top_venues = 5

for hood in grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = grouped[grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Barking and Dagenham [note 1]----
                 venue  freq
0                 Park  0.12
1  Martial Arts School  0.12
2          Bus Station  0.12
3                  Pub  0.12
4                 Pool  0.12


----Barnet----
                  venue  freq
0                  Café   0.4
1    Salon / Barbershop   0.2
2              Bus Stop   0.2
3          Home Service   0.2
4  Outdoor Supply Store   0.0


----Bexley----
                    venue  freq
0          Clothing Store  0.14
1             Coffee Shop  0.10
2                     Pub  0.10
3                Pharmacy  0.07
4  Furniture / Home Store  0.07


----Brent----
                 venue  freq
0          Coffee Shop  0.09
1                Hotel  0.09
2                  Bar  0.08
3  Sporting Goods Shop  0.06
4       Clothing Store  0.06


----Bromley----
                  venue  freq
0           Coffee Shop  0.12
1        Clothing Store  0.12
2  Gym / Fitness Center  0.05
3                   Bar  0.05
4          Burger Joint 

In [34]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [35]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = grouped['Neighborhood']

for ind in np.arange(grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Barking and Dagenham [note 1],Park,Bus Station,Martial Arts School,Gym / Fitness Center,Golf Course,Supermarket,Pool,Pub,Food & Drink Shop,Flower Shop
1,Barnet,Café,Home Service,Bus Stop,Salon / Barbershop,Farmers Market,Fast Food Restaurant,Film Studio,Fish & Chips Shop,Fish Market,Flea Market
2,Bexley,Clothing Store,Pub,Coffee Shop,Pharmacy,Fast Food Restaurant,Supermarket,Furniture / Home Store,Portuguese Restaurant,Chinese Restaurant,Sandwich Place
3,Brent,Hotel,Coffee Shop,Bar,Clothing Store,Sporting Goods Shop,Italian Restaurant,Sandwich Place,Grocery Store,Burger Joint,Indian Restaurant
4,Bromley,Coffee Shop,Clothing Store,Pizza Place,Bar,Gym / Fitness Center,Burger Joint,Cosmetics Shop,Café,Supermarket,Bookstore
5,Camden,Coffee Shop,Hotel,Café,Pub,Italian Restaurant,Burger Joint,Breakfast Spot,Tapas Restaurant,Garden,Plaza
6,Croydon,Pub,Coffee Shop,Portuguese Restaurant,Italian Restaurant,Asian Restaurant,Nightclub,Restaurant,Food Court,Sushi Restaurant,Supermarket
7,Ealing,Coffee Shop,Clothing Store,Bakery,Italian Restaurant,Pub,Park,Café,Bus Stop,Vietnamese Restaurant,Bar
8,Enfield,Clothing Store,Coffee Shop,Pub,Café,Supermarket,Gift Shop,Department Store,Optical Shop,Shopping Mall,Bookstore
9,Greenwich [note 2],Pub,Fast Food Restaurant,Coffee Shop,Clothing Store,Grocery Store,Supermarket,Hotel,Pharmacy,Plaza,Discount Store


In [36]:
from sklearn.cluster import KMeans
kclusters = 9
grouped_clustering = grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=1).fit(grouped_clustering)
kmeans.labels_

array([7, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 8, 0, 3, 3, 5, 3, 3, 3, 3, 3,
       8, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 1, 6, 1, 1, 1, 1,
       2, 3, 3, 3, 3, 3, 3, 3, 3])

In [37]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
merged1 = df1
merged1 = merged1.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
merged1 

,Neighborhood,longtitude,latitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS 02,48.8679033789,2.34410716666,1,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Bar,Bakery,Italian Restaurant,Thai Restaurant,Salad Place,Mexican Restaurant
1,PARIS 03,48.8630541318,2.35936105897,1,Coffee Shop,Bakery,French Restaurant,Bistro,Japanese Restaurant,Burger Joint,Cocktail Bar,Italian Restaurant,Art Gallery,Gourmet Shop
2,PARIS 08,48.8725272666,2.31258256042,1,French Restaurant,Hotel,Art Gallery,Theater,Cocktail Bar,Spa,Fast Food Restaurant,Mediterranean Restaurant,Furniture / Home Store,Modern European Restaurant
3,PARIS 14,48.8289932116,2.32710088326,1,French Restaurant,Hotel,Bistro,Japanese Restaurant,Food & Drink Shop,Café,Brasserie,Fast Food Restaurant,Bakery,Pizza Place
4,PARIS 16,48.8603987604,2.2620995594,6,Lake,Plaza,Park,Art Museum,Bus Station,Bus Stop,Boat or Ferry,French Restaurant,Pool,Dive Bar
5,PARIS 17,48.8873371665,2.30748555949,1,Hotel,French Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,Café,Restaurant,Plaza,Bistro,Breakfast Spot
6,PARIS 19,48.886868623,2.38469432787,1,French Restaurant,Bar,Hotel,Bistro,Supermarket,Brewery,Seafood Restaurant,Beer Bar,Concert Hall,Burger Joint
7,PARIS 06,48.8489680919,2.33267089859,1,French Restaurant,Chocolate Shop,Bistro,Bakery,Plaza,Restaurant,Pub,Fountain,Italian Restaurant,Athletics & Sports
8,PARIS 12,48.8351562307,2.41980703497,7,Zoo Exhibit,Pool,Monument / Landmark,Supermarket,Bistro,Park,Zoo,Dessert Shop,Farmers Market,French Restaurant
9,PARIS 13,48.8287176845,2.36246822852,1,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Hotel,Italian Restaurant,Creperie,Furniture / Home Store


In [42]:
merged1.loc[merged1['Cluster Labels'] == 1, merged1.columns[[0] + list(range(4, merged1.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,PARIS 02,French Restaurant,Cocktail Bar,Wine Bar,Hotel,Bar,Bakery,Italian Restaurant,Thai Restaurant,Salad Place,Mexican Restaurant
1,PARIS 03,Coffee Shop,Bakery,French Restaurant,Bistro,Japanese Restaurant,Burger Joint,Cocktail Bar,Italian Restaurant,Art Gallery,Gourmet Shop
2,PARIS 08,French Restaurant,Hotel,Art Gallery,Theater,Cocktail Bar,Spa,Fast Food Restaurant,Mediterranean Restaurant,Furniture / Home Store,Modern European Restaurant
3,PARIS 14,French Restaurant,Hotel,Bistro,Japanese Restaurant,Food & Drink Shop,Café,Brasserie,Fast Food Restaurant,Bakery,Pizza Place
5,PARIS 17,Hotel,French Restaurant,Bakery,Italian Restaurant,Japanese Restaurant,Café,Restaurant,Plaza,Bistro,Breakfast Spot
6,PARIS 19,French Restaurant,Bar,Hotel,Bistro,Supermarket,Brewery,Seafood Restaurant,Beer Bar,Concert Hall,Burger Joint
7,PARIS 06,French Restaurant,Chocolate Shop,Bistro,Bakery,Plaza,Restaurant,Pub,Fountain,Italian Restaurant,Athletics & Sports
9,PARIS 13,Vietnamese Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,French Restaurant,Juice Bar,Hotel,Italian Restaurant,Creperie,Furniture / Home Store
10,PARIS 18,Bar,French Restaurant,Restaurant,Pizza Place,Bistro,Café,Plaza,Italian Restaurant,Bakery,Convenience Store
11,PARIS 01,French Restaurant,Japanese Restaurant,Hotel,Coffee Shop,Plaza,Ramen Restaurant,Café,Italian Restaurant,Bakery,Art Museum
